Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [13]:
from itertools import combinations
import numpy as np
from copy import deepcopy
import random

In [14]:
def tour_length(tour, dist_matrix):
    return sum(dist_matrix[tour[i], tour[(i+1) % len(tour)]] for i in range(len(tour)))

def random_tour(n):
    t = list(range(n))
    random.shuffle(t)
    return t

## Simple Test Problem

In [15]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')

In [16]:
# Genetic Algorithm per TSP (permetti di personalizzare parametri)
import math

def order_crossover(p1, p2):
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    child = [-1]*n
    child[a:b] = p1[a:b]
    fill = [x for x in p2 if x not in child]
    k = 0
    for i in range(n):
        if child[i] == -1:
            child[i] = fill[k]; k += 1
    return child

def swap_mutation(tour, rate=0.2):
    if random.random() < rate:
        i, j = random.sample(range(len(tour)), 2)
        tour[i], tour[j] = tour[j], tour[i]
    return tour

def tournament_selection(pop, fitnesses, k=3):
    selected = []
    pop_idx = list(range(len(pop)))
    for _ in range(len(pop)):
        contenders = random.sample(pop_idx, k)
        best = min(contenders, key=lambda i: fitnesses[i])
        selected.append(deepcopy(pop[best]))
    return selected

def genetic_algorithm_tsp(dist_matrix, pop_size=100, generations=500,
                          cx_rate=0.9, mut_rate=0.2, tournament_k=3, seed=None):
    if seed is not None:
        random.seed(seed); np.random.seed(seed)
    n = len(dist_matrix)
    pop = [random_tour(n) for _ in range(pop_size)]
    best_sol = None
    best_len = math.inf

    for gen in range(generations):
        fitnesses = [tour_length(p, dist_matrix) for p in pop]
        # track best
        idx = int(np.argmin(fitnesses))
        if fitnesses[idx] < best_len:
            best_len = fitnesses[idx]; best_sol = deepcopy(pop[idx])

        # selection
        mates = tournament_selection(pop, fitnesses, k=tournament_k)
        # create offspring
        newpop = []
        for i in range(0, pop_size, 2):
            p1 = mates[i]
            p2 = mates[(i+1) % pop_size]
            if random.random() < cx_rate:
                c1 = order_crossover(p1, p2)
                c2 = order_crossover(p2, p1)
            else:
                c1, c2 = p1[:], p2[:]
            newpop.append(swap_mutation(c1, mut_rate))
            if len(newpop) < pop_size:
                newpop.append(swap_mutation(c2, mut_rate))
        pop = newpop

    return best_sol, best_len

# Esempio di esecuzione su 'problem'
best_ga, best_ga_len = genetic_algorithm_tsp(test_problem, pop_size=100, generations=1000, seed=0)
print("GA best length:", best_ga_len)


GA best length: 3101.0000000000005


In [17]:
# Evolution Strategy (1 + lambda) con mutazioni swap
import math

def es_1_plus_lambda(dist_matrix, lam=200, generations=1000, swap_mutations=5, seed=None):
    if seed is not None:
        random.seed(seed); np.random.seed(seed)
    n = len(dist_matrix)
    parent = random_tour(n)
    parent_fit = tour_length(parent, dist_matrix)
    best = deepcopy(parent); best_fit = parent_fit

    for g in range(generations):
        offspring = []
        for _ in range(lam):
            child = parent[:]
            # applica più swap per aumentare esplorazione
            for _ in range(swap_mutations):
                i, j = random.sample(range(n), 2)
                child[i], child[j] = child[j], child[i]
            offspring.append((child, tour_length(child, dist_matrix)))
        # prendi il migliore tra parent e figli
        cand, cand_fit = min(offspring + [(parent, parent_fit)], key=lambda x: x[1])
        parent, parent_fit = cand, cand_fit
        if parent_fit < best_fit:
            best, best_fit = deepcopy(parent), parent_fit

    return best, best_fit

# Esempio
best_es, best_es_len = es_1_plus_lambda(test_problem, lam=200, generations=500, swap_mutations=3, seed=0)
print("ES best length:", best_es_len)


ES best length: 2823.79


In [18]:
# PSO discreto per TSP usando sequenze di swap parziali
import math

def swap_sequence(a, b):
    # sequenza di swap che trasforma a in b (greedy)
    a = a[:]
    seq = []
    pos = {val:i for i,val in enumerate(a)}
    for i in range(len(a)):
        if a[i] != b[i]:
            j = pos[b[i]]
            # scambia pos i e j
            seq.append((i,j))
            val_i, val_j = a[i], a[j]
            a[i], a[j] = a[j], a[i]
            pos[val_i], pos[val_j] = j, i
    return seq

def apply_swap_seq(tour, seq, fraction=1.0):
    tour = tour[:]
    k = max(1, int(len(seq)*fraction))
    for i,j in seq[:k]:
        tour[i], tour[j] = tour[j], tour[i]
    return tour

def pso_tsp(dist_matrix, n_particles=50, iterations=500,
            c1=0.6, c2=0.6, frac_max=0.9, frac_min=0.1, seed=None):
    if seed is not None:
        random.seed(seed); np.random.seed(seed)
    n = len(dist_matrix)
    particles = [random_tour(n) for _ in range(n_particles)]
    pbest = [p[:] for p in particles]
    pbest_fit = [tour_length(p, dist_matrix) for p in pbest]
    gbest_idx = int(np.argmin(pbest_fit))
    gbest = pbest[gbest_idx][:]
    gbest_fit = pbest_fit[gbest_idx]

    for it in range(iterations):
        # decay fraction to focus search
        frac = frac_max - (frac_max-frac_min) * (it / max(1, iterations-1))
        for i in range(n_particles):
            cur = particles[i]
            # costruisci swap sequences verso pbest e gbest
            seq_p = swap_sequence(cur, pbest[i])
            seq_g = swap_sequence(cur, gbest)
            # combina le due sequenze probabilisticamente
            if random.random() < c1 and seq_p:
                cur = apply_swap_seq(cur, seq_p, fraction=frac * random.random())
            if random.random() < c2 and seq_g:
                cur = apply_swap_seq(cur, seq_g, fraction=frac * random.random())
            # piccolo rumore casuale
            if random.random() < 0.2:
                i1, i2 = random.sample(range(n), 2)
                cur[i1], cur[i2] = cur[i2], cur[i1]
            particles[i] = cur
            fit = tour_length(cur, dist_matrix)
            if fit < pbest_fit[i]:
                pbest[i], pbest_fit[i] = cur[:], fit
                if fit < gbest_fit:
                    gbest, gbest_fit = cur[:], fit
    return gbest, gbest_fit

# Esempio
best_pso, best_pso_len = pso_tsp(test_problem, n_particles=60, iterations=500, seed=0)
print("PSO best length:", best_pso_len)


PSO best length: 2982.42


## Common tests

In [19]:
problem = np.load('lab2/problem_r2_100.npy')

In [20]:
# Negative values?
np.any(problem < 0)

np.True_

In [21]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [22]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [23]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False